<a href="https://colab.research.google.com/github/Akash743/Mathematical-Optimization/blob/main/Optimizer_weekly_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyomo
!apt-get install -y -qq glpk-utils  #glpk is the solver
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.environ import * # for using pyomo functions like exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 28.6 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
P

In [5]:
model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,6)
model.j = pyo.RangeSet(1,13)

In [ ]:
x[i].pprint()

{Member of x} : Size=78, Index=x_index
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    (6, 13) :     0 :   0.0 :  None : False : False : NonNegativeReals


In [6]:
import pandas as pd

model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,6)
model.j = pyo.RangeSet(1,13)

# Parameters
S = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Spends',header=0,index_col=0,usecols='A:G',nrows=13)
C = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='CPP',header=0,index_col=0,usecols='A:G',nrows=13)
T = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Transformation',header=0,index_col=0,usecols='A:H',nrows=13)

"""
i = M1, M2, M3...
j = W1, W2, W3...
"""

'\ni = M1, M2, M3...\nj = W1, W2, W3...\n'

In [7]:
## for avoiding initial no spends terms in optimization
import numpy as np
CPP_check = np.where(C==0,0,1) 
CPP_check

array([[1, 0, 1, 0, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1],
       [1, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0]])

In [8]:

C = C.replace(0,1)

T = T.fillna(0)

In [133]:
C.head()

,1,2,3,4,5,6
Period,,,,,,
1,0.010282,1.0,0.000756,1.000000,1.000000,1.000000
2,0.011041,1.0,0.000773,0.000182,0.435862,1.000000
3,0.015013,1.0,0.000512,0.000169,0.338902,168.386464
4,0.017389,1.0,0.000545,0.000149,0.313671,168.386464
5,0.016280,1.0,1.000000,0.000118,0.270023,168.386464


In [9]:
spends = {}
for i in range(1,len(S.columns)+1):
  for j in range(1,len(S.index)+1):
    spends[(i,j)] = S.loc[j,i]
# cpp = {} 
# for i in range(1,len(C.columns)+1):
#   for j in range(1,len(C.index)+1):
#     cpp[(i,j)] = C.loc[j,i]

In [10]:
T['Ret_rate'] = (1/2)**(1/T['Half Life'])

In [136]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Adjustment_factor,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,4.176702e+04,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,1.511057e+05,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,3.617750e+06,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,5.104543e+05,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000e+00,0.000000


In [11]:
## Decision variable----------------trying without initializing
# Spends
model.x = pyo.Var(model.i,model.j,domain = pyo.NonNegativeReals,initialize = spends) #initialize = spends
x = model.x

#Adstock
model.ads = pyo.Var(model.i,model.j,domain=pyo.NonNegativeReals,initialize=0) #,initialize=0
ads = model.ads




In [10]:
#x.pprint()

In [12]:
#Saturation
model.satu = pyo.Var(model.i,model.j,domain=pyo.NonNegativeReals)
satu = model.satu

In [152]:
C

,1,2,3,4,5,6
Period,,,,,,
1,0.010282,1.000000,0.000756,1.000000,1.000000,1.000000
2,0.011041,1.000000,0.000773,0.000182,0.435862,1.000000
3,0.015013,1.000000,0.000512,0.000169,0.338902,168.386464
4,0.017389,1.000000,0.000545,0.000149,0.313671,168.386464
5,0.016280,1.000000,1.000000,0.000118,0.270023,168.386464
6,0.015711,1.000000,1.000000,1.000000,0.256921,1.000000
7,0.015745,1.000000,1.000000,1.000000,0.256921,140.839250
8,0.015555,1.000000,1.000000,0.000142,0.256921,140.839250
9,1.000000,1.000000,1.000000,0.000134,0.256921,140.839250


In [13]:
## working code with model with everything
import numpy as np
# def adstest():
  ##Adstocking
#ads2 = pd.DataFrame()
#ads2 = pd.DataFrame()
#ads2 = pd.DataFrame()
def objective_func(model):
  for j in range(1,S.shape[1]+1): #j for var
    #print(j)
    if T.loc[j,'Type'] == 'Media':
      ads[j,1] = (x[j,1]/C.loc[1,j])*CPP_check[0,j-1]
      

      for i in range(2,S.shape[0]+1): #i for week
        ads[j,i] = ads[j,i-1]*T.loc[j,'Ret_rate'] + (x[j,i]/C.loc[i,j])*CPP_check[i-1,j-1]
        
    else: # for promo
      for i in range(1,S.shape[0]+1): 
        ads[j,i] = (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
        

    
    ##Saturation transformation
  #satu = pd.DataFrame()
  for i in range(1,S.shape[1]+1): #j for week, i for var
    if T.loc[i,'Type'] == 'Media':
      for j in range(1,S.shape[0]+1):
      
        satu[i,j] = (T.loc[i,'Max Adstock']/\
      (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
      (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))-T.loc[i,'Adjustment_factor'])*T.loc[i,'Coefficient']   #*\
      #T.loc[i,'Coefficient'] 
    
    else: 
      for j in range(1,S.shape[0]+1): # j for week
        satu[i,j] = ads[i,j]
  print(satu[i,j].value)    

  return sum(sum(satu[i,j] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

# ##Replacing small -ve numbers with 0
  #satu[satu<0] = 0



In [14]:
model.Objf = pyo.Objective(rule = objective_func, sense = pyo.maximize)

26.363953750656577


In [15]:
#Spends sum for constraints
spends_sum = []
for i in range(S.shape[1]):
  spends_sum.append(sum(S[S.columns[i]]))
  print(sum(S[S.columns[i]]))

11641.779999999986
2713.2644939999964
31989.04778999995
20531.32999999
20155.63736263736
172021.8294117647


In [16]:
LB = .85
UB = 1.15

In [17]:
##Constraints

# def Constraint1(model,j):
#   return sum(x[1,i].value for i in model.i) <= sums1
# model.Const1 = pyo.Constraint(model.i,rule = Constraint1)

def Constraint1(model,i):
  return sum(x[1,j] for j in model.j) <= spends_sum[0]*UB
model.Const1 = pyo.Constraint(model.j,rule = Constraint1)

def Constraint11(model,i):
  return sum(x[1,j] for j in model.j) >= spends_sum[0]*LB
model.Const11 = pyo.Constraint(model.j,rule = Constraint11)

def Constraint2(model,i):
  return sum(x[2,j] for j in model.j) <= spends_sum[1]*UB
model.Const2 = pyo.Constraint(model.j, rule = Constraint2)

def Constraint22(model,i):
  return sum(x[2,j] for j in model.j) >= spends_sum[1]*LB
model.Const22 = pyo.Constraint(model.j, rule = Constraint22)

def Constraint3(model,i):
  return sum(x[3,j] for j in model.j) <= spends_sum[2]*UB
model.Const3 = pyo.Constraint(model.j, rule = Constraint3)

def Constraint33(model,i):
  return sum(x[3,j] for j in model.j) >= spends_sum[2]*LB
model.Const33 = pyo.Constraint(model.j, rule = Constraint33)


def Constraint4(model,i):
  return sum(x[4,j] for j in model.j) <= spends_sum[3]*UB
model.Const4 = pyo.Constraint(model.j, rule = Constraint4)

def Constraint44(model,i):
  return sum(x[4,j] for j in model.j) >= spends_sum[3]*LB
model.Const44 = pyo.Constraint(model.j, rule = Constraint44)


def Constraint5(model,i):
  return sum(x[5,j] for j in model.j) <= spends_sum[4]*UB
model.Const5 = pyo.Constraint(model.j, rule = Constraint5)

def Constraint55(model,i):
  return sum(x[5,j] for j in model.j) >= spends_sum[4]*LB
model.Const55 = pyo.Constraint(model.j, rule = Constraint55)


def Constraint6(model,i):
  return sum(x[6,j] for j in model.j) <= spends_sum[5]*UB
model.Const6 = pyo.Constraint(model.j, rule = Constraint6)

def Constraint66(model,i):
  return sum(x[6,j] for j in model.j) >= spends_sum[5]*LB
model.Const66 = pyo.Constraint(model.j, rule = Constraint66)


def Constraint7(model,i,j):
  return sum(sum(x[i,j] for j in model.j) for i in model.i) <= sum(spends_sum)
model.Const7 = pyo.Constraint(model.i,model.j, rule = Constraint7)





# print('objective function =', model.Obj())
# print('s1 = ',model.s1())
# print('s2 = ',model.s2())

# print('sum total = ',model.s1()+model.s2())

In [18]:
optm = SolverFactory('couenne',executable='/content/couenne')
results = optm.solve(model)

In [19]:
print('Objective Function = ', model.Objf())

Objective Function =  9990000000000.0


In [20]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 156
  Sense: unknown
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.8054258823394775
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [24]:
for i in model.i:
  for j in model.j:
    print('M',i, 'week',j, '=', ads[i,j]())

M 1 week 1 = 163533.0
M 1 week 2 = 306032.5
M 1 week 3 = 371480.25
M 1 week 4 = 334416.125
M 1 week 5 = 231775.0625
M 1 week 6 = 132665.53125
M 1 week 7 = 77361.765625
M 1 week 8 = 47078.8828125
M 1 week 9 = 23539.44140625
M 1 week 10 = 11769.720703125
M 1 week 11 = 5884.8603515625
M 1 week 12 = 2942.43017578125
M 1 week 13 = 1471.215087890625
M 2 week 1 = 0
M 2 week 2 = 0.0
M 2 week 3 = 0.0
M 2 week 4 = 0.0
M 2 week 5 = 0.0
M 2 week 6 = 0.0
M 2 week 7 = 0.0
M 2 week 8 = 0.0
M 2 week 9 = 0.0
M 2 week 10 = 1669588.0000000005
M 2 week 11 = 4738171.0
M 2 week 12 = 6521021.75
M 2 week 13 = 8401198.4375
M 3 week 1 = 8122073.0
M 3 week 2 = 24635642.3707414
M 3 week 3 = 29561342.200289562
M 3 week 4 = 25182838.650645223
M 3 week 5 = 15864194.25602706
M 3 week 6 = 9993816.141394915
M 3 week 7 = 6295709.662661306
M 3 week 8 = 3966048.564006765
M 3 week 9 = 2498454.0353487288
M 3 week 10 = 1573927.4156653264
M 3 week 11 = 991512.1410016912
M 3 week 12 = 624613.5088371822
M 3 week 13 = 393481.853

In [23]:
sum2=0
for i in model.i:
  for j in model.j:
    print('M',i, 'week',j, '=', x[i,j]())
    sum2 += x[i,j]()
print(sum2)    

M 1 week 1 = 0.0
M 1 week 2 = 0.0
M 1 week 3 = 0.0
M 1 week 4 = 0.0
M 1 week 5 = 0.0
M 1 week 6 = 0.0
M 1 week 7 = 0.0
M 1 week 8 = 0.0
M 1 week 9 = 0.0
M 1 week 10 = 0.0
M 1 week 11 = 0.0
M 1 week 12 = 0.0
M 1 week 13 = 9895.512999999988
M 2 week 1 = 0.0
M 2 week 2 = 0.0
M 2 week 3 = 0.0
M 2 week 4 = 0.0
M 2 week 5 = 0.0
M 2 week 6 = 2306.274819899997
M 2 week 7 = 0.0
M 2 week 8 = 0.0
M 2 week 9 = 0.0
M 2 week 10 = 0.0
M 2 week 11 = 0.0
M 2 week 12 = 0.0
M 2 week 13 = 0.0
M 3 week 1 = 0.0
M 3 week 2 = 0.0
M 3 week 3 = 0.0
M 3 week 4 = 0.0
M 3 week 5 = 0.0
M 3 week 6 = 0.0
M 3 week 7 = 0.0
M 3 week 8 = 0.0
M 3 week 9 = 27190.690621499958
M 3 week 10 = 0.0
M 3 week 11 = 0.0
M 3 week 12 = 0.0
M 3 week 13 = 0.0
M 4 week 1 = 0.0
M 4 week 2 = 0.0
M 4 week 3 = 0.0
M 4 week 4 = 0.0
M 4 week 5 = 0.0
M 4 week 6 = 0.0
M 4 week 7 = 0.0
M 4 week 8 = 0.0
M 4 week 9 = 0.0
M 4 week 10 = 0.0
M 4 week 11 = 0.0
M 4 week 12 = 17451.6304999915
M 4 week 13 = 0.0
M 5 week 1 = 0.0
M 5 week 2 = 0.0
M 5 week 3

In [123]:
model.pprint()

instance = model.create()

opt = SolverFactory("cplex")

solver_manager = SolverManagerFactory('neos')

results = solver_manager.solve(instance, opt=opt, tee=True)

model.pprint()

instance.display()

4 Set Declarations
    Const7_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   78 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13)}
    ads_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   78 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1

AttributeError: ignored

In [121]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 156
  Sense: unknown
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.15123748779296875
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



M 1 week 1 = 0.0
M 1 week 2 = 0.0
M 1 week 3 = 0.0
M 1 week 4 = 0.0
M 1 week 5 = 0.0
M 1 week 6 = 0.0
M 1 week 7 = 0.0
M 1 week 8 = 0.0
M 1 week 9 = 0.0
M 1 week 10 = 0.0
M 1 week 11 = 0.0
M 1 week 12 = 0.0
M 1 week 13 = 0.0
M 2 week 1 = 0.0
M 2 week 2 = 0.0
M 2 week 3 = 0.0
M 2 week 4 = 0.0
M 2 week 5 = 0.0
M 2 week 6 = 0.0
M 2 week 7 = 0.0
M 2 week 8 = 0.0
M 2 week 9 = 0.0
M 2 week 10 = 0.0
M 2 week 11 = 0.0
M 2 week 12 = 0.0
M 2 week 13 = 0.0
M 3 week 1 = 0.0
M 3 week 2 = 0.0
M 3 week 3 = 0.0
M 3 week 4 = 0.0
M 3 week 5 = 0.0
M 3 week 6 = 0.0
M 3 week 7 = 0.0
M 3 week 8 = 0.0
M 3 week 9 = 0.0
M 3 week 10 = 0.0
M 3 week 11 = 0.0
M 3 week 12 = 0.0
M 3 week 13 = 0.0
M 4 week 1 = 0.0
M 4 week 2 = 0.0
M 4 week 3 = 0.0
M 4 week 4 = 0.0
M 4 week 5 = 0.0
M 4 week 6 = 0.0
M 4 week 7 = 0.0
M 4 week 8 = 0.0
M 4 week 9 = 0.0
M 4 week 10 = 0.0
M 4 week 11 = 0.0
M 4 week 12 = 0.0
M 4 week 13 = 0.0
M 5 week 1 = 0.0
M 5 week 2 = 0.0
M 5 week 3 = 0.0
M 5 week 4 = 0.0
M 5 week 5 = 0.0
M 5 week 6 = 0.

In [ ]:
sum(spends_sum)

259052.88905839197

In [ ]:
# for i in model.x:
#   print(i)

In [ ]:
## working code with everything
import numpy as np
# def adstest():
  ##Adstocking
ads2 = pd.DataFrame()
for j in range(1,S.shape[1]+1):
  #print(j)
  if T.loc[j,'Type'] == 'Media':
    ads2.loc[1,j] = (S.loc[1,j]/C.loc[1,j])*CPP_check[i-1,j-1]
    #ads2.fillna(0,inplace=True)
    #print(ads2)

    for i in range(2,S.shape[0]+1):
      ads2.loc[i,j] = ads2.loc[i-1,j]*T.loc[j,'Ret_rate'] + (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
      #ads2.fillna(0,inplace=True)
      #print('media')
  else: # for promo
    for i in range(1,S.shape[0]+1): 
      ads2.loc[i,j] = (S.loc[i,j]/C.loc[i,j])*CPP_check[i-1,j-1]
      

  
  ##Saturation transformation
satu = pd.DataFrame()
for i in range(1,S.shape[1]+1):
  if T.loc[i,'Type'] == 'Media':
    for j in range(1,S.shape[0]+1):
    
      satu.loc[j,i] = (T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads2.loc[j,i]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))-T.loc[i,'Adjustment_factor'])*T.loc[i,'Coefficient']   #*\
    #T.loc[i,'Coefficient'] 
  
  else:
    satu[i] = ads2[i]
# ##Replacing small -ve numbers with 0
  satu[satu<0] = 0

print(ads2)    
print(satu)


                1             2             3             4            5  \
1   163533.000000  0.000000e+00  8.122073e+06  0.000000e+00     0.000000   
2   306032.500000  0.000000e+00  2.463564e+07  7.066104e+07  3551.230769   
3   371480.250000  0.000000e+00  2.956134e+07  8.092103e+07  4913.868132   
4   334416.125000  0.000000e+00  2.518284e+07  7.635885e+07  5458.923077   
5   231775.062500  0.000000e+00  1.586419e+07  6.740568e+07  6755.626374   
6   132665.531250  0.000000e+00  9.993816e+06  5.349992e+07  7274.307692   
7    77361.765625  0.000000e+00  6.295710e+06  4.246292e+07  7274.307692   
8    47078.882812  0.000000e+00  3.966049e+06  3.440765e+07  7274.307692   
9    23539.441406  0.000000e+00  2.498454e+06  2.852018e+07  7274.307692   
10   11769.720703  1.669588e+06  1.573927e+06  2.596281e+07  7274.307692   
11    5884.860352  4.738171e+06  9.915121e+05  2.192770e+07  7274.307692   
12    2942.430176  6.521022e+06  6.246135e+05  1.740403e+07  6140.241758   
13    1471.2

In [ ]:
adstest()

NameError: ignored

In [ ]:
import numpy as np
def adstest():
  ads2 = pd.DataFrame()
  for j in range(1,S.shape[1]+1):
    ads2.loc[1,j] = S.loc[1,j]/C.loc[1,j]
    ads2.fillna(0,inplace=True)

    for i in range(2,S.shape[0]+1):
      ads2.loc[i,j] = ads2.loc[i-1,j]*T.loc[j,'Ret_rate'] + S.loc[i,j]/C.loc[i,j]
      ads2.fillna(0,inplace=True)
  return ads2    


In [ ]:
ads2 = pd.DataFrame()
for j in range(1,S.shape[1]+1):
  #ads2.loc[1,j] = S.loc[1,j]/C.loc[1,j]
  print(S.loc[1,j])
  print(C.loc[1,j])

1681.499999999998
0.010282328337399779
0.0
0.0
6141.681549999989
0.0007561716756300995
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
for j in range(1,S.shape[0]+1):
  for i in range(1,S.shape[1]+1):

    a = sum(sum(T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
    T.loc[i,'Coefficient'] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

In [ ]:
##Final code ready

sum(sum(T.loc[i,'Max Adstock']/\
    (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
    (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
    T.loc[i,'Coefficient'] for i in range(1,S.shape[1]+1)) for j in range(1,S.shape[0]+1))

In [ ]:
fo i in range(a.values


AttributeError: ignored

In [ ]:
for i in model.i:
  print(i)

1
2
3
4
5
6


In [ ]:
ads[1,1].value

0

In [ ]:
a[1,1]

TypeError: ignored

In [ ]:
a = sum(sum(i) for i in range(1,5)) for j in range(1,3) 

SyntaxError: ignored

In [ ]:
S.shape[0]

13

In [ ]:
for i in range(1,S.shape[1]+1):
  for j in range(1,S.shape[0]+1):

    T.loc[i,'Max Adstock']/\
        (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
        (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
        T.loc[i,'Coefficient']

In [ ]:
sum(sum(T.loc[i,'Max Adstock']/\
        (1+exp(-(10*T.loc[i,'Steepness']/T.loc[i,'Max Adstock'])*\
        (ads[i,j]-(T.loc[i,'Saturation']*T.loc[i,'Max Adstock']))))*\
        T.loc[i,'Coefficient']) for j in range(1,S.shape[0]+1)) for i in range(1,S.shape[1]+1)

TypeError: ignored

In [ ]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000


In [ ]:
# for i in ads:
#   print(ads[i],ads[i].value)

In [ ]:
S.head()

,1,2,3,4,5,6
Period,,,,,,
1,1681.50,0.0,6141.681550,0.00,0.000000,0.000000
2,2476.10,0.0,15080.853226,12833.30,1547.846154,0.000000
3,3279.79,0.0,7192.357366,4191.13,1665.318681,39133.014118
4,2585.39,0.0,3574.155648,1806.90,1712.307692,39133.014118
5,1051.12,0.0,0.000000,800.00,1824.175824,5590.430588


In [ ]:
T.head(8)

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient,Ret_rate
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835,0.500000
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069,0.250000
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006,0.629961
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022,0.793701
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272,0.000000
6,Media,3.8,7.525281e+02,0.6,1.5,0.251257,0.833262


In [ ]:
## Objective Function
def objective_rule(model):
  for i in range(1,len(model.typ)+1): ##i for var
    if model.typ[i] == "Media":
      


0.0

In [ ]:

from pyomo.environ import *

CPP = Spends/Raw Execution(RE)

RE = Spends/CPP

Adstocked execution = RE * Adstock factor(AF)
= (Spends/CPP)*AF

Avg adst execution = (Spends*AF/CPP)/Exec. periods

Now apply transformation on Avg adst execution to get TCP

Lift = TCP * Exec. periods * coeffcient

Put lift in Optimization equation



In [ ]:
## Defining the model
import numpy as np
#import math
from pyomo.environ import *
model = pyo.ConcreteModel()


sp1 = 33150.8411764706
sp2 = 233150.268146738


## Decision Variables
model.s1 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp1,1.1*sp1)) 
s1 = model.s1
model.s2 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp2,1.1*sp2))
s2 = model.s2


expre = ((1953.64352941177/(1+exp(-(10*3/1953.64352941177)*((s1*1/47/1)-0.179872867132035*1953.64352941177))))-8.81752993140885)*0.00133831972924945*47 + ((11441.0307432077/(1+exp(-(10*1/11441.0307432077)*((s2*1/1/154)-0.202998593210211*11441.0307432077))))-1328.19344200206)*0.00245568243410747*154

## Objective Function
model.Obj = pyo.Objective(expr =expre, sense = pyo.maximize)

## Constraint
model.Const1 = pyo.Constraint(expr = s1+s2<=266301.109323209)



optm = SolverFactory('couenne',executable='/content/couenne')
results = optm.solve(model)



print('objective function =', model.Obj())
print('s1 = ',model.s1())
print('s2 = ',model.s2())

print('sum total = ',model.s1()+model.s2())

objective function = 1065.7698788838547
s1 =  29835.757357158283
s2 =  236465.35226438538
sum total =  266301.10962154367


In [ ]:
# model.s3 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp3,1.3*sp3))
# s3 = model.s3
# model.s4 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp4,1.3*sp4))
# s4 = model.s4
# model.s5 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp5,1.3*sp5))
# s5 = model.s5
# model.s6 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp6,1.3*sp6))
# s6 = model.s6